# Train FOMO-like YOLOv8 on Kaggle (COCO 2017)

This notebook allows you to train the model on Kaggle using the attached COCO 2017 dataset.

## Instructions
1. Add the **COCO 2017 Dataset** to your Kaggle notebook.
   - Dataset URL: [https://www.kaggle.com/code/derekxue/coco-2017-dataset/data](https://www.kaggle.com/code/derekxue/coco-2017-dataset/data)
   - Ensure it is mounted at `/kaggle/input/coco-2017-dataset/coco2017`.
2. Upload the `tiny-vin` code to the notebook (or clone it).
3. Run the cells below.

In [ ]:
# @title 1. Setup Environment
import os
import sys

# Assuming code is in the current directory or uploaded
# If you uploaded the folder as a dataset, copy it to working dir to allow edits
if not os.path.exists('requirements.txt'):
    # Example: if code is in /kaggle/input/tiny-vin
    # !cp -r /kaggle/input/tiny-vin/* .
    pass

# Install dependencies
!pip install -r requirements.txt

In [ ]:
# @title 2. Prepare Labels
# @markdown The COCO dataset on Kaggle is read-only. We need to convert the JSON annotations to YOLO format and save them in the working directory.

import sys
sys.path.append('.')
from src.datasets.download.coco import convert_coco_json_to_yolo
from pathlib import Path

# Define paths
KAGGLE_INPUT = Path('/kaggle/input/coco-2017-dataset/coco2017')
WORKING_DIR = Path('/kaggle/working/data/coco')

# Input Images (Read-only)
train_img_dir = KAGGLE_INPUT / 'train2017'
val_img_dir = KAGGLE_INPUT / 'val2017'

# Output Labels (Writable)
train_lbl_dir = WORKING_DIR / 'train/labels'
val_lbl_dir = WORKING_DIR / 'val/labels'

print("Converting Train Labels...")
convert_coco_json_to_yolo(
    KAGGLE_INPUT / 'annotations/instances_train2017.json',
    train_lbl_dir,
    train_img_dir
)

print("\nConverting Val Labels...")
convert_coco_json_to_yolo(
    KAGGLE_INPUT / 'annotations/instances_val2017.json',
    val_lbl_dir,
    val_img_dir
)

print("\nLabels prepared!")

In [ ]:
# @title 3. Train Model
# @markdown We point the training script to the read-only images and writable labels.

!PYTHONPATH=. python scripts/train_coco.py \
    --train-image-dir /kaggle/input/coco-2017-dataset/coco2017/train2017 \
    --train-label-dir /kaggle/working/data/coco/train/labels \
    --val-image-dir /kaggle/input/coco-2017-dataset/coco2017/val2017 \
    --val-label-dir /kaggle/working/data/coco/val/labels \
    --batch-size 32 \
    --epochs 100 \
    --output-dir checkpoints_kaggle

In [ ]:
# @title 4. Visualize Predictions
!PYTHONPATH=. python scripts/visualize_predictions.py \
    --checkpoint checkpoints_kaggle/best_model.pth \
    --image-dir /kaggle/input/coco-2017-dataset/coco2017/val2017 \
    --num-samples 5 \
    --output-dir predictions_kaggle